In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType

In [ ]:
import base64
import itertools
import json
import re
from typing import List, Type

import requests
import os

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool


def strip_markdown_code(md_string: str) -> str:
    stripped_string = re.sub(r"^`{1,3}.*?\n", "", md_string, flags=re.DOTALL)
    stripped_string = re.sub(r"`{1,3}$", "", stripped_string)
    return stripped_string


def head_file(path: str, n: int) -> List[str]:
    try:
        with open(path, "r") as f:
            return [str(line) for line in itertools.islice(f, n)]
    except Exception:
        return []


def file_to_base64(path: str) -> str:
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode()


def make_input_files(file_info_arr: any) -> List[dict]:
    files = []
    for file_info in file_info_arr:
        files.append(
            {
                "pathname": file_info["target_path"],
                "contentsBasesixtyfour": file_to_base64(file_info["source_path"]),
            }
        )
    return files


class BearlyInterpreterToolArguments(BaseModel):
    python_code: str = Field(
        ...,
        example="print('Hello World')",
        description=(
            "The pure python script to be evaluated. "
            "The contents will be in main.py. "
            "It should not be in markdown format."
        ),
    )
    file_info_arr_json: str = Field(
        ...,
        example='["source_path":"/home/user/1.txt","description":"It is a file for test.","target_path":"1.txt"]',
        description=(
            "An array string in JSON format. "
            "Each element represents the data of a file, including source_path, target_path and description. "
        ),
    )


class FileInfo(BaseModel):
    source_path: str
    description: str
    target_path: str


class BearlyInterpreterTool(BaseTool):
    endpoint = "https://exec.bearly.ai/v1/interpreter"
    name = "bearly_interpreter"
    args_schema: Type[BaseModel] = BearlyInterpreterToolArguments
    description = """Then evaluates python code in a sandbox environment. \
The environment resets on every execution. \
You must send the whole script every time and print your outputs. \
Script should be pure python code that can be evaluated. \
It should be in python format NOT markdown. \
The code should NOT be wrapped in backticks. \
All python packages including requests, matplotlib, scipy, numpy, pandas, \
etc are available. \
If you have any files outputted write them to "output/" relative to the execution \
path. Output can only be read from the directory, stdout, and stdin. \
Do not use things like plot.show() as it will \
not work instead write them out `output/` and a link to the file will be returned. \
print() any output and results so you can capture the output."""

    def _run(self, python_code: str, file_info_arr_json: str) -> dict:
        file_info_arr = json.loads(file_info_arr_json)
        script = strip_markdown_code(python_code)
        resp = requests.post(
            "https://exec.bearly.ai/v1/interpreter",
            data=json.dumps(
                {
                    "fileContents": script,
                    "inputFiles": make_input_files(file_info_arr=file_info_arr),
                    "outputDir": "output/",
                    "outputAsLinks": True,
                }
            ),
            headers={"Authorization": os.getenv("BEARLY_API_KEY")},
        ).json()
        return {
            "stdout": base64.b64decode(resp["stdoutBasesixtyfour"]).decode()
            if resp["stdoutBasesixtyfour"]
            else "",
            "stderr": base64.b64decode(resp["stderrBasesixtyfour"]).decode()
            if resp["stderrBasesixtyfour"]
            else "",
            "fileLinks": resp["fileLinks"],
            "exitCode": resp["exitCode"],
        }

    async def _arun(self, python_code: str, file_info_arr_json: str) -> str:
        file_info_arr = json.loads(file_info_arr_json)
        script = strip_markdown_code(python_code)
        resp = requests.post(
            "https://exec.bearly.ai/v1/interpreter",
            data=json.dumps(
                {
                    "fileContents": script,
                    "inputFiles": make_input_files(file_info_arr=file_info_arr),
                    "outputDir": "output/",
                    "outputAsLinks": True,
                }
            ),
            headers={"Authorization": os.getenv("BEARLY_API_KEY")},
        ).json()
        return {
            "stdout": base64.b64decode(resp["stdoutBasesixtyfour"]).decode()
            if resp["stdoutBasesixtyfour"]
            else "",
            "stderr": base64.b64decode(resp["stderrBasesixtyfour"]).decode()
            if resp["stderrBasesixtyfour"]
            else "",
            "fileLinks": resp["fileLinks"],
            "exitCode": resp["exitCode"],
        }

In [ ]:

from dotenv import load_dotenv

load_dotenv(dotenv_path="env")

bearly_tool = BearlyInterpreterTool()

In [ ]:
tools = [bearly_tool]
llm = ChatOpenAI(model="gpt-4", temperature=0.9,verbose=True)
agent = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_MULTI_FUNCTIONS, verbose=True, handle_parsing_errors=True
)

In [ ]:
filePath=input("Input your file path:")
file_info=[{
    "source_path":filePath, 
    "target_path":filePath, 
    "description":"",
},]
file_info_str=json.dumps(file_info)
while True:
    user_input = input("Your prompt: ")
    if user_input == ":exit":
        break
    user_input="""Uploaded files info:
"""+file_info_str+"\n"+user_input
    print(user_input)
    res = await agent.arun(user_input)
    print(f"AI:{res}")